# Write & Rewrite Barium forcing data in NEMO

The notebook writes all necessary variables into netCDF4 to run the model. The correspond script is `_script/Create_netCDF4.py`. 

In [1]:
import glob
import scipy.io
import numpy as np
import netCDF4 as nc
from __future__ import print_function
% matplotlib inline

#ORCA2

## River source

In [2]:
# Ba
Ba_name=glob.glob('_data/Exchange/Ba_ORCA2_SRC.mat')
Ba_Mat=scipy.io.loadmat(Ba_name[0])
Ba_ORCA2_2deg=Ba_Mat['Ba_ORCA2_2deg']
nav_lon=Ba_Mat['nav_lon']
nav_lat=Ba_Mat['nav_lat']
# d18O
d18O_name=glob.glob('_data/Exchange/d18O_ORCA2_SRC.mat')
d18O_Mat=scipy.io.loadmat(d18O_name[0])
d18O_ORCA2_2deg=d18O_Mat['d18O_ORCA2_2deg']

** Get `time_counter` from Xianmin's original file. **

In [6]:
#ANHA4_name=glob.glob('_data/NEMO/ANHA4_runoff_monthly_DaiTrenberth_fewPTs.nc')
#ANH4_obj=nc.Dataset(ANHA4_name[0])
#time_counter=ANH4_obj.variables['time_counter'][:]
time_counter = [  15.,   46.,   75.,  106.,  136.,  167.,  197.,  228.,  259.,  289.,  320.,  350.]

In [7]:
print(time_counter)

[15.0, 46.0, 75.0, 106.0, 136.0, 167.0, 197.0, 228.0, 259.0, 289.0, 320.0, 350.0]


In [5]:
nc_obj = nc.Dataset('_data/Exchange/TRC_ORCA2_SRC.nc', 'w') # format='NETCDF4'
nc_obj.description = 'River sources of Barium concentration in Arctic'
print(nc_obj.file_format)

NETCDF4


In [6]:
Ba_ORCA2_2deg.shape

(12L, 149L, 182L)

In [7]:
time=nc_obj.createDimension('time', None)
lat=nc_obj.createDimension('lat', 149)
lon=nc_obj.createDimension('lon', 182)

In [8]:
print(time.isunlimited())

True


In [9]:
time_counter_obj=nc_obj.createVariable('time_counter', np.float32, ('time',), zlib=True)
BaVar_obj=nc_obj.createVariable('Ba_ORCA2_2deg', np.float64, ('time', 'lat', 'lon'), zlib=True)
d18OVar_obj=nc_obj.createVariable('d18O_ORCA2_2deg', np.float64, ('time', 'lat', 'lon'), zlib=True)
nav_lat_obj=nc_obj.createVariable('nav_lat', np.float64, ('lat', 'lon'), zlib=True)
nav_lon_obj=nc_obj.createVariable('nav_lon', np.float64, ('lat', 'lon'), zlib=True)

In [10]:
time_counter_obj[:]=time_counter
BaVar_obj[:]=Ba_ORCA2_2deg
d18OVar_obj[:]=d18O_ORCA2_2deg
nav_lat_obj[:]=nav_lat
nav_lon_obj[:]=nav_lon

In [11]:
nc_obj.close()

## Initial field

In [12]:
MAT = scipy.io.loadmat('_data/Exchange/NEMO_ORCA2_Ba.mat')
Ba_orca = MAT['Ba_ini_orca'][:]
MAT = scipy.io.loadmat('_data/Exchange/NEMO_ORCA2_d18O.mat')
d18O_orca = MAT['d18O_ini_orca'][:]
#Ba_orca[np.isnan(Ba_orca)] = -999
d18O_name=glob.glob('_data/Exchange/NEMO_ORCA2_d18O_Grid.mat')
d18O_Mat=scipy.io.loadmat(d18O_name[0])
d18O_grid=d18O_Mat['d18O_grid_orca']

boundary_name=glob.glob('_data/Exchange/TRC_BOUND.mat')
Boundary_Mat=scipy.io.loadmat(boundary_name[0])
Ba_boundary=Boundary_Mat['Ba']
d18O_boundary=Boundary_Mat['d18O']

In [13]:
ini_obj = nc.Dataset('_data/Exchange/TRC_ORCA2_Nomask.nc', 'w') # format='NETCDF4'
#ini_obj.description = 'Barium Initial Field'
print(ini_obj.file_format)

NETCDF4


In [14]:
time=ini_obj.createDimension('time', None)
lev = ini_obj.createDimension('lev', 31)
lat=ini_obj.createDimension('lat', 149)
lon=ini_obj.createDimension('lon', 182)

In [15]:
time_counter_obj=ini_obj.createVariable('time_counter', np.float32, ('time',), zlib=True)
BaVar_obj=ini_obj.createVariable('Ba', np.float64, ('time', 'lev', 'lat', 'lon'), zlib=True)
d18OVar_obj=ini_obj.createVariable('d18O', np.float64, ('time', 'lev', 'lat', 'lon'), zlib=True)
d18OGVar_obj=ini_obj.createVariable('d18O_grid', np.float64, ('time', 'lev', 'lat', 'lon'), zlib=True)
BaBVar_obj=ini_obj.createVariable('Ba_boundary', np.float64, ('time', 'lev', 'lat', 'lon'), zlib=True)
d18OBVar_obj=ini_obj.createVariable('d18O_boundary', np.float64, ('time', 'lev', 'lat', 'lon'), zlib=True)
nav_lat_obj=ini_obj.createVariable('nav_lat', np.float64, ('lat', 'lon'), zlib=True)
nav_lon_obj=ini_obj.createVariable('nav_lon', np.float64, ('lat', 'lon'), zlib=True)

In [16]:
#
Ba = np.empty([1, 31, 149, 182])
Ba[0, :, :, :] = Ba_orca

BaB = np.empty([1, 31, 149, 182])
BaB[0, :, :, :] = Ba_boundary
#
d18O = np.empty([1, 31, 149, 182])
d18O[0, :, :, :] = d18O_orca
#
d18OB = np.empty([1, 31, 149, 182])
d18OB[0, :, :, :] = d18O_boundary
#
d18OG = np.empty([1, 31, 149, 182])
d18OG[0, :, :, :] = d18O_grid

In [17]:
time_counter_obj[:]=3600.0
BaVar_obj[:]=Ba
BaBVar_obj[:]=BaB
d18OVar_obj[:]=d18O
d18OBVar_obj[:]=d18OB
d18OGVar_obj[:]=d18OG
nav_lat_obj[:]=nav_lat
nav_lon_obj[:]=nav_lon

In [18]:
BaVar_obj.units='1e-6 mol/L'

In [19]:
ini_obj.close()

In [20]:
#test = nc.Dataset('_data/Exchange/TRC_ORCA2_Nomask.nc', 'r')
#test.variables['d18O_grid'][:]

# ANHA4

In [21]:
Ba_name=glob.glob('_data/Exchange/Ba_ANHA4_SRC.mat')
Ba_Mat=scipy.io.loadmat(Ba_name[0])
Ba_ANHA4=Ba_Mat['Ba_ANHA4']
nav_lon=Ba_Mat['nav_lon']
nav_lat=Ba_Mat['nav_lat']

d18O_name=glob.glob('_data/Exchange/d18O_ANHA4_SRC.mat')
d18O_Mat=scipy.io.loadmat(d18O_name[0])
d18O_ANHA4=d18O_Mat['d18O_ANHA4']

In [22]:
d18O_ANHA4.shape

(12L, 544L, 800L)

In [23]:
np.transpose(Ba_ANHA4, [0, 2, 1]).shape

(12L, 800L, 544L)

In [24]:
nav_lon.shape

(544L, 800L)

In [25]:
boundary_name=glob.glob('_data/Exchange/Ba_boundary_ANHA4.mat')
Boundary_Mat=scipy.io.loadmat(boundary_name[0])
Ba_boundary=Boundary_Mat['Ba_boundary']

In [26]:
Ba_boundary.shape

(50L, 544L, 800L)

In [27]:
nc_obj = nc.Dataset('_data/Exchange/TRC_ANHA4_SRC.nc', 'w') # format='NETCDF4'
nc_obj.description = 'River sources of Barium concentration in Arctic'
print(nc_obj.file_format)

NETCDF4


In [28]:
time=nc_obj.createDimension('time', None)
lat=nc_obj.createDimension('lon', 544)
lon=nc_obj.createDimension('lat', 800)

In [29]:
time_counter_obj=nc_obj.createVariable('time_counter', np.float32, ('time',), zlib=True)
BaVar_obj=nc_obj.createVariable('Ba_ANHA4', np.float64, ('time', 'lat', 'lon'), zlib=True)
d18OVar_obj=nc_obj.createVariable('d18O_ANHA4', np.float64, ('time', 'lat', 'lon'), zlib=True)
#BaB_obj = nc_obj.createVariable('Ba_boundary', np.float64, ('lat', 'lon'), zlib=True)
nav_lat_obj=nc_obj.createVariable('nav_lat', np.float64, ('lat', 'lon'), zlib=True)
nav_lon_obj=nc_obj.createVariable('nav_lon', np.float64, ('lat', 'lon'), zlib=True)

In [30]:
Ba_ANHA4.shape

(12L, 544L, 800L)

In [31]:
time_counter_obj[:]=time_counter
BaVar_obj[:]=np.transpose(Ba_ANHA4, [0, 2, 1])
d18OVar_obj[:]=np.transpose(d18O_ANHA4, [0, 2, 1])
#BaB_obj[:]=Ba_boundary.T
nav_lat_obj[:]=nav_lat.T
nav_lon_obj[:]=nav_lon.T

In [32]:
nc_obj.close()

## Initial field

In [33]:
MAT = scipy.io.loadmat('_data/Exchange/NEMO_ANHA4_Ba.mat')
Ba_ANHA4 = MAT['Ba_ini_ANHA4'][:]

MAT = scipy.io.loadmat('_data/Exchange/NEMO_ANHA4_d18O_Grid.mat')
d18O_ANHA4 = MAT['d18O_grid_ANHA']

In [34]:
ini_obj = nc.Dataset('_data/Exchange/TRC_ANHA4_Nomask.nc', 'w') # format='NETCDF4'
ini_obj.description = 'Barium Initial Field'
print(ini_obj.file_format)

NETCDF4


In [35]:
time=ini_obj.createDimension('time', None)
lev = ini_obj.createDimension('lev', 50)
lat=ini_obj.createDimension('lat', 800)
lon=ini_obj.createDimension('lon', 544)

In [36]:
time_counter_obj=ini_obj.createVariable('time_counter', np.float32, ('time',), zlib=True)
BaVar_obj=ini_obj.createVariable('Ba', np.float64, ('time', 'lev', 'lat', 'lon'), zlib=True)
d18OVar_obj=ini_obj.createVariable('d18O', np.float64, ('time', 'lev', 'lat', 'lon'), zlib=True)
BaBVar_obj=ini_obj.createVariable('Ba_boundary', np.float64, ('time', 'lev', 'lat', 'lon'), zlib=True)
d18OBVar_obj=ini_obj.createVariable('d18O_boundary', np.float64, ('time', 'lev', 'lat', 'lon'), zlib=True)
nav_lat_obj=ini_obj.createVariable('nav_lat', np.float64, ('lat', 'lon'), zlib=True)
nav_lon_obj=ini_obj.createVariable('nav_lon', np.float64, ('lat', 'lon'), zlib=True)

In [37]:
Ba_ANHA4.shape

(50L, 544L, 800L)

In [38]:
Ba_boundary=np.transpose(Ba_boundary, [0, 2, 1])

In [39]:
Ba = np.empty([1, 50, 800, 544])
Ba[0, :, :, :] = np.transpose(Ba_ANHA4, [0, 2, 1])

d18O = np.empty([1, 50, 800, 544])
d18O[0, :, :, :] = np.transpose(d18O_ANHA4, [0, 2, 1])

d18OB = np.empty([1, 50, 800, 544])
d18OB[0, :, :, :] = Ba_boundary
#
BaB = np.empty([1, 50, 800, 544])
BaB[0, :, :, :] = Ba_boundary

In [40]:
time_counter_obj[:]=3600.0
BaVar_obj[:]=Ba
d18OVar_obj[:]=d18O
nav_lat_obj[:]=nav_lat
nav_lon_obj[:]=nav_lon
BaBVar_obj[:]=BaB
d18OBVar_obj[:]=d18OB

In [41]:
ini_obj.close()

In [42]:
test_obj = nc.Dataset('_data/Exchange/TRC_ANHA4_Nomask.nc', 'r')

In [43]:
B = test_obj.variables['Ba_boundary'][:]

In [44]:
B[0, :, 799]

array([[ 65.83534541,  65.83534541,  65.83534541, ...,  50.0016647 ,
         50.0016647 ,  49.97100188],
       [ 65.71576117,  65.71576117,  65.71576117, ...,  45.67335344,
         45.67335344,  45.69154488],
       [ 64.85842617,  64.85842617,  64.85842617, ...,  40.13345059,
         40.13345059,  40.13422968],
       ..., 
       [ 62.39230504,  62.39230504,  62.39230504, ...,  49.07489906,
         49.07489906,  49.05929393],
       [ 62.39230504,  62.39230504,  62.39230504, ...,  49.07489906,
         49.07489906,  49.05929393],
       [ 62.39230504,  62.39230504,  62.39230504, ...,  49.07489906,
         49.07489906,  49.05929393]])